In [1]:
import torch
import pyro
from pyro.optim import Adam
import congas as cg
from congas.models import MixtureGaussian,MixtureGaussianEXP, MixtureGaussianOld, MixtureGaussianNorm
from pyro.infer import TraceEnum_ELBO


In [2]:
data = cg.simulation_data

interface = cg.Interface()

In [3]:
interface.set_model(MixtureGaussian)
interface.set_optimizer(Adam)
interface.set_loss(TraceEnum_ELBO)
interface.initialize_model(data)
interface.set_model_params({'K': 2})

In [4]:
ll = interface.run(steps = 100, MAP = False)



Running MixtureGaussian on 1000 cells wiht 32 segments for 100 steps


ELBO: 108.532046875  : 100%|██████████| 100/100 [00:02<00:00, 45.58it/s]


Done!


In [5]:
import numpy as np

lr = interface.learned_parameters()
lr

Computing assignment probabilities


{'param_mixture_weights': array([0.62456715, 0.37543282], dtype=float32),
 'param_cnv_probs': array([[1.993137  , 1.9512253 , 0.97713286, 2.030496  , 1.9912943 ,
         1.9611102 , 1.9545438 , 1.0185893 , 1.99126   , 2.047696  ,
         0.96906763, 1.9631591 , 1.9941092 , 1.9966292 , 2.9202852 ,
         1.9832568 , 1.9447126 , 2.071764  , 1.9576341 , 3.01876   ,
         1.0150421 , 2.049042  , 1.987777  , 2.031935  , 1.9752474 ,
         2.0179265 , 1.9542118 , 2.0840023 , 1.9630741 , 1.9711814 ,
         0.9981359 , 1.9648991 ],
        [2.0567508 , 2.0744908 , 1.0469588 , 2.0200458 , 2.1062887 ,
         2.0600398 , 2.0227973 , 2.0686007 , 2.0288637 , 2.0763376 ,
         1.0171864 , 0.99372894, 1.9904584 , 1.961207  , 3.011462  ,
         2.0043912 , 2.0322027 , 2.0405326 , 1.9781868 , 3.0026674 ,
         0.99886215, 1.925858  , 2.0107944 , 2.0725794 , 1.9756205 ,
         1.9786181 , 2.0418289 , 2.0876122 , 2.0635183 , 2.0131507 ,
         1.0011046 , 2.0834    ]], dtype=floa